In [1]:
#=================================================================================
# sentencepiece vocab에 기존 단어 중복 검사하면서 신규 단어 추가하는 예제
# => 'albert-base-v2' 모델의 sentencepiece vocab(사이즈:30,000개)에 신규 단어 추가해 봄
#
#=================================================================================
# sentencepiece_model_pb2 사용을 위해서는 google-api-python-client 설치해야 함
#!pip install --upgrade google-api-python-client

import sentencepiece as spm
import sentencepiece.sentencepiece_model_pb2 as spmodel

In [2]:
# 기존 sentenctpiece 모델 불러옴
# => albert-base-v2 모델이 vocab 경로(spiece.model) 지정.
smodel_path = '../data11/model/bert/albert-base-v2/spiece.model'
m = spmodel.ModelProto()
m.ParseFromString(open(smodel_path, 'rb').read())


760289

In [10]:
# 신규 단어 추가시, 중복 검사를 위해..
# 기존 sentenctpiece vocab 목록을 리스트에 저장해 둠.
vocab_list = []
count = 0
for i, piece in enumerate(m.pieces):
    #print(piece.piece)
    vocab_list.append(piece.piece)
    count += 1


In [11]:
vocab_list[50:100]

['▁are',
 '▁my',
 '▁not',
 '▁one',
 '▁or',
 '▁me',
 '▁which',
 '▁have',
 'a',
 '▁they',
 '?',
 '▁him',
 'e',
 '▁has',
 '▁first',
 '▁all',
 '▁their',
 '▁also',
 'ing',
 'ed',
 '▁out',
 '▁up',
 '▁who',
 ';',
 '▁been',
 '▁after',
 '▁when',
 '▁into',
 '▁new',
 'm',
 '▁there',
 '▁two',
 '▁its',
 '▁would',
 '▁over',
 '▁time',
 '▁so',
 '▁said',
 '▁about',
 '▁other',
 '▁no',
 '▁more',
 '▁can',
 'y',
 '▁then',
 '▁we',
 'th',
 '▁back',
 '▁what',
 're']

In [12]:
print(m.pieces[2])
print(type(m.pieces[0]))

piece: "[CLS]"
score: 0.0
type: CONTROL

<class 'sentencepiece_model_pb2.SentencePiece'>


In [13]:
# 새로운 단어들 추가
# => 기존 cat, w 단어들은 기존에 존재하는 단어이므로, 중복 검사 하여 추가하지 않음.
new_vocab_list = ['문서중앙화', '보안파일서버', '엠파워', 'Mpower', 'EZis-C', '모코엠시스', 'M드라이브', 'cat', 'w']

# for문을 돌면서, 기존 vocab에 있는 단어인지 검사 후,
# 없는 단어들만 추가함.
for idx, vocab in enumerate(new_vocab_list):
    
    # 기존 vocab에 없는 단어들만 추가 
    if vocab not in vocab_list:
        print(f'idx:{idx}, vocab:{vocab}')
        new_piece = type(m.pieces[0])()
        new_piece.piece = vocab
        new_piece.score = 0.0
        new_piece.type = 1

        m.pieces.append(new_piece)


idx:0, vocab:문서중앙화
idx:1, vocab:보안파일서버
idx:2, vocab:엠파워
idx:3, vocab:Mpower
idx:4, vocab:EZis-C
idx:5, vocab:모코엠시스
idx:6, vocab:M드라이브


In [14]:
# 새로운 모델에 serialize 함.(저장함)
new_smodel_path = '../data11/model/bert/albert-base-v2/spiece_new.model'
with open(new_smodel_path, 'wb') as f:
    f.write(m.SerializeToString())

In [15]:
# 기존 spmodel 테스트
text = "모코엠시스에서는 문서중앙화 및 보안파일서버 솔루션인 엠파워를 출시하였다."
sp_old = spm.SentencePieceProcessor(model_file=smodel_path)
print(sp_old.encode(text, out_type=str))

['▁', '모코엠시스에서는', '▁', '문서중앙화', '▁', '및', '▁', '보안파일서버', '▁', '솔루션인', '▁', '엠파워를', '▁', '출시하였다', '.']


In [16]:
# 새로운 spmodel 테스트 
# => 추가된 단어들 별루 분리가 잘 된다.
sp_new = spm.SentencePieceProcessor(model_file=new_smodel_path)
print(sp_new.encode(text, out_type=str))

['▁', '모코엠시스', '에서는', '▁', '문서중앙화', '▁', '및', '▁', '보안파일서버', '▁', '솔루션인', '▁', '엠파워', '를', '▁', '출시하였다', '.']


In [22]:
# 새로운 모델 vocba 출력해 봄
print('new_Model_size:{}'.format(sp_new.GetPieceSize()))
print('vocab:{}'.format(sp_new.PieceToId('엠파워')))
#print('idtoPiece:{}'.format(sp_new.IdToPiece(30002)))

new_Model_size:29956
vocab:29951


In [23]:
# 기존 모델에 단어들이 새로운 모델에 어떻게 적용되었는지 출력 해봄
print(sp_new.PieceToId('cat'))
print(sp_new.PieceToId('w'))


5782
499


In [20]:
# 신규 모델에 파일을 불러옴
import torch
from transformers import AlbertTokenizer, AlbertForMaskedLM, AlbertModel

vocab_path = '../data11/model/bert/albert-base-v2/newvocab' # 앞에서 만들어진 spiece_new.model 이 있는 경로 지정해줌
model_path = '../data11/model/bert/albert-base-v2'

tokenizer = AlbertTokenizer.from_pretrained(vocab_path)
print(len(tokenizer))

model = AlbertForMaskedLM.from_pretrained(model_path)

# resize_token_embeddings 으로 신규 tokenizer 사이즈로 지정 해줌.
model.resize_token_embeddings(len(tokenizer))
print(model)


Some weights of AlbertForMaskedLM were not initialized from the model checkpoint at ../data11/model/bert/albert-base-v2 and are newly initialized: ['predictions.dense.weight', 'predictions.decoder.bias', 'predictions.bias', 'predictions.LayerNorm.bias', 'predictions.dense.bias', 'predictions.LayerNorm.weight', 'predictions.decoder.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


30007
AlbertForMaskedLM(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30007, 128)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
           

In [21]:
model.num_parameters()

11222583

In [24]:
# text tokenizer 해봄.
token_ids = tokenizer.encode_plus(text, max_length=128, padding="max_length", return_tensors="pt")
print(token_ids)

{'input_ids': tensor([[    2,    13, 30005,     1,    13, 30000,    13,     1,    13, 30001,
            13,     1,    13, 30002,     1,    13,     1,     9,     3,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  